In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23).apply(lambda x: (x - x.mean()) 
                                                                / (x.std()))
train_loc = train.iloc[:,30:]

In [3]:
#all_features[numeric_feats] = all_features[numeric_feats].apply(lambda x: (x - x.mean()) 
                                                              #  / (x.std()))

In [4]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [5]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [6]:
feat_dim = train_data.shape[0]

In [7]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)


        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model.t7")

def predict(x_test):
    net = torch.load('./model.t7')
    a = net(x_test).detach().numpy()
    a = np.argmax(a,axis=1)
    with open('./ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')
    

In [8]:
# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
#predict(x_test)

epoch:0
error:36.57128779915912
epoch:1
error:36.48215657341527
epoch:2
error:36.40144930465718
epoch:3
error:36.30259540750323
epoch:4
error:36.199893374029756
epoch:5
error:36.073251030886155
epoch:6
error:35.92715071151358
epoch:7
error:35.72445524741268
epoch:8
error:35.49207632600259
epoch:9
error:35.22708552312419
epoch:10
error:34.952867278460545
epoch:11
error:34.6545520698577
epoch:12
error:34.357368814683056
epoch:13
error:34.00273134298189
epoch:14
error:33.66931547137775
epoch:15
error:33.32440067108668
epoch:16
error:32.945448940815005
epoch:17
error:32.560453387774906
epoch:18
error:32.143050614489006
epoch:19
error:31.719664658796894
epoch:20
error:31.266428484799484
epoch:21
error:30.81649215717982
epoch:22
error:30.34353775873221
epoch:23
error:29.881160050129367
epoch:24
error:29.36820322606727
epoch:25
error:28.868746462645536
epoch:26
error:28.379371159443725
epoch:27
error:27.834396224126777
epoch:28
error:27.34220367076326
epoch:29
error:26.806592618046572
epoch:3

epoch:241
error:1.8417434988478332
epoch:242
error:1.8656351698940814
epoch:243
error:1.975777333643273
epoch:244
error:1.8877081038264067
epoch:245
error:1.9207618387673835
epoch:246
error:1.912235709087969
epoch:247
error:1.8690357677069858
epoch:248
error:1.8402584416437582
epoch:249
error:1.9217594043095083
epoch:250
error:1.82023101451326
epoch:251
error:1.8133303307931758
epoch:252
error:1.8623144783918175
epoch:253
error:1.8247670399417852
epoch:254
error:1.8210698728573738
epoch:255
error:1.8242588610729302
epoch:256
error:1.7844421465677556
epoch:257
error:1.8293346488923028
epoch:258
error:1.7633351946555627
epoch:259
error:1.8585753052231566
epoch:260
error:1.836812363559185
epoch:261
error:1.782866128820343
epoch:262
error:1.7826736272538002
epoch:263
error:1.8003585045581338
epoch:264
error:1.9062345240742238
epoch:265
error:1.8406995055283797
epoch:266
error:2.2773247998868045
epoch:267
error:1.9648953890584573
epoch:268
error:1.9317983846923512
epoch:269
error:1.93817114

epoch:478
error:1.611881784140524
epoch:479
error:1.6646229495977523
epoch:480
error:1.6401414562782988
epoch:481
error:1.7055329277166882
epoch:482
error:1.664233208936368
epoch:483
error:1.714176074345084
epoch:484
error:1.7151918004123545
epoch:485
error:1.7367911394324063
epoch:486
error:1.608202303828428
epoch:487
error:1.6481136109819696
epoch:488
error:1.767122848530482
epoch:489
error:1.7994215793782342
epoch:490
error:1.6775739307082795
epoch:491
error:1.6655587693543823
epoch:492
error:1.6509190173229302
epoch:493
error:1.642899487336271
epoch:494
error:1.6857853305910413
epoch:495
error:1.6781512774902976
epoch:496
error:1.6092761435761642
epoch:497
error:1.6282199400671087
epoch:498
error:1.5804626543802556
epoch:499
error:1.6173536293256792
epoch:500
error:1.7166125851491754
epoch:501
error:1.6587902626738358
epoch:502
error:1.5986626589282826
epoch:503
error:2.0212092825234476
epoch:504
error:2.3266773692694858
epoch:505
error:1.8453459153662677
epoch:506
error:1.74746984

epoch:714
error:1.6069288562217012
epoch:715
error:1.5584912614711353
epoch:716
error:1.5178342673633571
epoch:717
error:1.5320806466283958
epoch:718
error:1.515400756993855
epoch:719
error:1.5244093249717012
epoch:720
error:1.5430286007741754
epoch:721
error:1.5637630250444696
epoch:722
error:1.5933185690895861
epoch:723
error:1.556616937358506
epoch:724
error:1.4839855751738358
epoch:725
error:1.471919406431921
epoch:726
error:1.523921675392141
epoch:727
error:1.5964974505781049
epoch:728
error:1.4987489766938875
epoch:729
error:1.5169530870997736
epoch:730
error:1.5258716420399419
epoch:731
error:1.5173320893636804
epoch:732
error:1.6040370689480918
epoch:733
error:1.4752746818907665
epoch:734
error:1.5363854806759378
epoch:735
error:1.538785670429738
epoch:736
error:1.4875377738923028
epoch:737
error:1.6549823953448415
epoch:738
error:1.5206546758772639
epoch:739
error:1.6032264778561611
epoch:740
error:1.5838645155340394
epoch:741
error:1.6495634525590233
epoch:742
error:1.5508760

epoch:950
error:1.5499600784282017
epoch:951
error:1.468551023811449
epoch:952
error:1.4843609453327133
epoch:953
error:1.4980253982050453
epoch:954
error:1.4837613321676908
epoch:955
error:1.544462334755013
epoch:956
error:1.4831045528278621
epoch:957
error:1.5356762726896023
epoch:958
error:1.5328301236052717
epoch:959
error:1.4702211604543984
epoch:960
error:1.492411111335705
epoch:961
error:1.5413990489569858
epoch:962
error:1.5115211950699385
epoch:963
error:1.4882420864327297
epoch:964
error:1.4833534309811611
epoch:965
error:1.5401630278238196
epoch:966
error:1.5015659110203752
epoch:967
error:1.500186974450194
epoch:968
error:1.6364025989448576
epoch:969
error:1.503650265554253
epoch:970
error:1.438878620734961
epoch:971
error:1.475723957491106
epoch:972
error:1.4708240898892302
epoch:973
error:1.5379886602522639
epoch:974
error:1.5113637512128073
epoch:975
error:1.5094816891372898
epoch:976
error:1.6142856059993531
epoch:977
error:1.6240663910899094
epoch:978
error:1.515266369

epoch:1181
error:1.5346303843588291
epoch:1182
error:1.4585442586210382
epoch:1183
error:1.469148584047542
epoch:1184
error:1.4373869309912677
epoch:1185
error:1.425793567573577
epoch:1186
error:1.4040628411020375
epoch:1187
error:1.4840629548027167
epoch:1188
error:1.4592670475218306
epoch:1189
error:1.4496197028116915
epoch:1190
error:1.4331882175776196
epoch:1191
error:1.487866084603412
epoch:1192
error:1.4314031169146184
epoch:1193
error:1.4719186168438714
epoch:1194
error:1.4020495494926424
epoch:1195
error:1.5331762790063066
epoch:1196
error:1.4825197839181758
epoch:1197
error:1.488643355079641
epoch:1198
error:1.5064240883732212
epoch:1199
error:1.4738096802231566
epoch:1200
error:1.5277009596337323
epoch:1201
error:1.4576722375788325
epoch:1202
error:1.4615134255235285
epoch:1203
error:1.5205633994987064
epoch:1204
error:1.495910881407665
epoch:1205
error:1.6700463140766495
epoch:1206
error:1.5122728828933538
epoch:1207
error:1.567822770961352
epoch:1208
error:1.511435445807729

KeyboardInterrupt: 

In [ ]:
        
        '''
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        '''
        